In [18]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Carregar dados
df = pd.read_csv("BASE.csv", sep=';')

# Tratar dados
df['Preço'] = df['Preço'].str.replace(',', '.')
df['Preço'] = pd.to_numeric(df['Preço'])
df['Data'] = pd.to_datetime(df['Data'])

# Filtrar os registros com data maior que 01/01/1990
df = df[df['Data'] > '1990-01-01']

# Limpar nulos
df = df.dropna()

# Gráfico 1: Série Temporal
fig_serie = go.Figure()
fig_serie.add_trace(
    go.Scatter(x=df["Data"], y=df["Preço"], mode="lines+markers", name="Preço", line=dict(color="blue"))
)

# Adicionar eventos econômicos
eventos = [
    {"Data": "1990-08-02", "descricao": "Início da Guerra do Golfo"},
    {"Data": "1991-02-28", "descricao": "Fim da Guerra do Golfo"},
    {"Data": "2020-05-01", "descricao": "Acordo de produção OPEP"},
    {"Data": "2020-09-01", "descricao": "Tensão política no Oriente Médio"},
    {"Data": "2008-07-11", "descricao": "Preço recorde do petróleo - Crise financeira"},
    {"Data": "2020-03-01", "descricao": "Impacto inicial da pandemia"},
    {"Data": "2014-06-20", "descricao": "Queda de preços devido ao excesso de oferta"}
]
for evento in eventos:
    fig_serie.add_annotation(
        x=evento["Data"],
        y=df.loc[df["Data"] == evento["Data"], "Preço"].values[0] if evento["Data"] in df["Data"].astype(str).values else None,
        text=evento["descricao"],
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40,
        bgcolor="yellow",
        bordercolor="black",
        font=dict(size=10),
    )

fig_serie.update_layout(
    title="Série Temporal do Preço com Eventos Econômicos",
    xaxis_title="Data",
    yaxis_title="Preço (USD)",
    template="plotly_white",
)
fig_serie.show()


<ipython-input-18-9eefdb7bfb3b>:11: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [19]:
# Converter a coluna 'Data' para o formato datetime (caso ainda não esteja)
df["Data"] = pd.to_datetime(df["Data"])

# Criar uma nova coluna para agrupar por mês/ano
df["Mes_Ano"] = df["Data"].dt.to_period("M").dt.to_timestamp()

# Agrupar por mês/ano e calcular a média dos preços
df_mensal = df.groupby("Mes_Ano")["Preço"].mean().reset_index()

# Obter os 5 meses com maiores altas e os 5 meses com maiores baixas
top = df_mensal.nlargest(3, "Preço")
bottom = df_mensal.nsmallest(3, "Preço")

# Adicionar uma sigla para os destaques
top["Sigla"] = ["A" + str(i+1) for i in range(len(top))]
bottom["Sigla"] = ["B" + str(i+1) for i in range(len(bottom))]

# Criar o gráfico de linha
fig = go.Figure()

# Linha principal de evolução dos preços
fig.add_trace(
    go.Scatter(
        x=df_mensal["Mes_Ano"],
        y=df_mensal["Preço"],
        mode="lines",
        name="Preço Médio Mensal",
        line=dict(color="blue", width=2),
        hovertemplate="Mês/Ano: %{x}<br>Preço Médio: %{y:.2f} USD"
    )
)

# Adicionar pontos vermelhos
fig.add_trace(
    go.Scatter(
        x=top["Mes_Ano"],
        y=top["Preço"],
        mode="markers+text",
        name="Maiores Altas (Meses)",
        marker=dict(size=10, color="red"),
        text=top["Sigla"],
        textposition="top center",
        hovertemplate="Mês/Ano: %{x}<br>Preço Máximo: %{y:.2f} USD"
    )
)

# Adicionar pontos verdes
fig.add_trace(
    go.Scatter(
        x=bottom["Mes_Ano"],
        y=bottom["Preço"],
        mode="markers+text",
        name="Maiores Baixas (Meses)",
        marker=dict(size=10, color="green"),
        text=bottom["Sigla"],
        textposition="bottom center",
        hovertemplate="Mês/Ano: %{x}<br>Preço Mínimo: %{y:.2f} USD"
    )
)

# Ajustar layout
fig.update_layout(
    title="Evolução do Preço Médio Mensal do Petróleo com Destaques",
    template="plotly_white",
    xaxis_title="Mês/Ano",
    yaxis_title="Preço Médio (USD)",
    margin=dict(l=40, r=40, t=60, b=200),
)

# Exibir o gráfico
fig.show()

# Análise do Gráfico de Altas e Baixas do Preço do Petróleo

O gráfico destaca os cinco meses com os preços mais altos e mais baixos do petróleo ao longo do período analisado:

# Altas Históricas (Top 3 Meses):
Os meses de maior preço estão concentrados em períodos de **crises econômicas e instabilidades geopolíticas**, como em **2008-07**, durante o pico da crise financeira global, e em **2012-03**, reflexo de **tensões no Oriente Médio**.
Esses meses indicam impactos diretos de eventos externos sobre o mercado, gerando aumento na demanda ou restrições de oferta.

# Baixas Históricas (Top 3 Meses):
Os meses de menores preços são marcados por excesso de oferta e crises econômicas, como **1998-12**, associado à **crise asiática**, e **2020-04**, durante o auge da pandemia de **COVID-19**, quando os preços chegaram a níveis extremamente baixos devido à queda abrupta na demanda global.
A proximidade das datas reflete períodos prolongados de desvalorização.

# Conclusão:
Este gráfico ajuda a entender como fatores externos, como crises financeiras e tensões geopolíticas, afetam diretamente os preços do petróleo. Ele também destaca a volatilidade do mercado, permitindo análises mais direcionadas para estratégias de mitigação de riscos e previsão de preços.

In [20]:
# Gráfico 3: Projeção com Média Móvel
df["Média Móvel (30 dias)"] = df["Preço"].rolling(window=30).mean()

fig_projecao = px.line(
    df,
    x="Data",
    y=["Preço", "Média Móvel (30 dias)"],
    title="Projeção - Média Móvel (30 dias)",
    labels={"value": "Preço (USD)", "variable": "Indicador"},
)
fig_projecao.update_layout(template="plotly_white")
fig_projecao.show()
